In [3]:
import vaex
import vaex.jupyter
import pandas as pd
import requests
import json

# Data preprocessing

## Fetch travels data from Portic API

In [6]:
portic_api_url = "http://data.portic.fr/api/travels"
api_params = 'travel_id,outdate_fixed,distance_dep_dest_miles,departure,departure_latitude,departure_longitude,departure_admiralty,departure_province,departure_uncertainity,destination,destination_latitude,destination_longitude,destination_admiralty,destination_province,destination_uncertainity,travel_uncertainity,tonnage,commodity_id'
def get_portic_api_data():
    headers = {'Accept': 'application/json'}
    params = {'params': f'{api_params}',
              'shortenfields': 'false',
              'both_to': 'false',
              'date': '1787'}
    response = requests.request(
        "GET", portic_api_url, headers=headers, params=params)
    data = json.loads(response.text)
    df = pd.DataFrame(data)
    return df

df = get_portic_api_data()
df.to_json("../static/data/travels_init.json", orient="records", force_ascii=False)

In [7]:
df.shape

(39451, 18)

### Process

In [8]:
# Step: Keep rows where (departure_admiralty is not missing) and (destination_admiralty is not missing)
df = df.loc[(df['departure_admiralty'].notna()) & (df['destination_admiralty'].notna())]

# Step: Replace missing values
df[['distance_dep_dest_miles', 'tonnage']] = df[['distance_dep_dest_miles', 'tonnage']].fillna(0)

# Step: Change data type of ['distance_dep_dest_miles', 'tonnage'] to Integer
for column_name in ['distance_dep_dest_miles', 'tonnage']:
    df[column_name] = df[column_name].astype('float')

# Step: Keep rows where outdate_fixed starts with 1787
df = df.loc[df['outdate_fixed'].str.startswith('1787', na=False)]

# Step: Change data type of outdate_fixed to Datetime
df['outdate_fixed_datetime'] = pd.to_datetime(df['outdate_fixed'], format='%Y-%m-%d')

# Step: Extract datetime attribute(s) month number from 'outdate_fixed_datetime'
df['outdate_fixed_datetime_month_number'] = df['outdate_fixed_datetime'].dt.month

# Step: Extract datetime attribute(s) month name from 'outdate_fixed_datetime'
df['outdate_fixed_datetime_month_name'] = df['outdate_fixed_datetime'].dt.month_name()

# Step: Extract datetime attribute(s) week from 'outdate_fixed_datetime'
df['outdate_fixed_datetime_week'] = df['outdate_fixed_datetime'].dt.week

# Step: Change data type of ['outdate_fixed_datetime_month_number', 'outdate_fixed_datetime_week'] to String/Text
for column_name in ['outdate_fixed_datetime_month_number', 'outdate_fixed_datetime_week']:
    df[column_name] = df[column_name].astype('string')
    
# Step: Drop columns
df = df.drop(columns=['outdate_fixed'])

# Step: Change data type of commodity_id to String/Text
df['commodity_id'] = df['commodity_id'].astype('string')
#df['commodity_id'] = df['commodity_id'].astype(object).replace(np.nan, 'None')

# Step: Manipulate strings of 'commodity_id' via Find '.0' and Replace with ''
df["commodity_id"] = df["commodity_id"].str.replace('.0', '', regex=False)
df["commodity_id"] = df["commodity_id"].apply(lambda x: x.zfill(8) if (pd.notnull(x)) else x)

C:\Users\BUNICE\AppData\Local\Temp\ipykernel_11472\3882798018.py:24: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  df['outdate_fixed_datetime_week'] = df['outdate_fixed_datetime'].dt.week


In [9]:
df.shape

(25432, 21)

## Fetch products data from Portic new products API

In [10]:
portic_products_api_url = "http://data.portic.fr/api/cargo_categories/"
def get_portic_products_api_data():
    headers = {'Accept': 'application/json'}
    response = requests.request(
        "GET", portic_products_api_url, headers=headers)
    data = json.loads(response.text)
    df = pd.DataFrame(data)
    return df

df_products = get_portic_products_api_data()

### Save raw data

In [11]:
df_products.to_json("../static/data/products.json", orient="records", force_ascii=False)

## Merge

In [13]:
# Step: Left Join with df1 where commodity_id=record_id
df_travels = pd.merge(df, df_products[['record_id', 'commodity_standardized_fr', 'category_portic_fr']], how='left', left_on=['commodity_id'], right_on=['record_id']).drop(columns=['record_id'])

## Save

In [14]:
df.shape

(25432, 21)

In [15]:
df_travels.to_json("../static/data/travels.json", orient="records", force_ascii=False)

# Vaex stuffs

https://vaex.io/blog/8-incredibly-powerful-Vaex-features-you-might-have-not-known-about

### Read data

In [8]:
df_vaex_json = vaex.from_json('../static/data/travels.json', orient='records', copy_index=False)
df_vaex_json

#,travel_id,distance_dep_dest_miles,departure,departure_latitude,departure_longitude,departure_admiralty,departure_province,departure_uncertainity,destination,destination_latitude,destination_longitude,destination_admiralty,destination_province,destination_uncertainity,travel_uncertainity,tonnage,commodity_id,outdate_fixed_datetime,outdate_fixed_datetime_month_number,outdate_fixed_datetime_month_name,outdate_fixed_datetime_week,commodity_standardized_fr,category_portic_fr
0,0000076N- 29,20,Granville,48.838204,-1.597853,Granville,Normandie,0,Saint-Malo,48.65,-2.016667,Saint-Malo,Bretagne,0,0,19.0,94.0,-5754326400000,8,August,35,Lest,Sur lest et vides
1,0005962N- 07,20,Tonnay-Charente,45.95,-0.9,Marennes,Saintonge,0,île de Ré,46.2,-1.416667,La Rochelle,Aunis,-3,-2,20.0,nan,-5764780800000,4,April,17,--,--
2,0000624N- 05,32,Le Havre,49.5,0.133333,Le Havre,Normandie,0,Caudebec,49.524483,0.725362,Caudebec et Quilleboeuf,Normandie,-5,-3,13.0,nan,-5767372800000,3,March,13,--,--
3,0000359N- 04,51,Honfleur,49.420164,0.231886,Honfleur,Normandie,0,Rouen,49.433333,1.083333,Rouen,Normandie,0,0,112.0,nan,-5766854400000,4,April,14,--,--
4,0000624N- 16,20,Le Havre,49.5,0.133333,Le Havre,Normandie,0,Pont-Audemer,49.35,0.516667,Caudebec et Quilleboeuf,Normandie,-5,-3,13.0,nan,-5761152000000,6,June,23,--,--
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"25,427",0021805N- 08,48,Toulon,43.116667,5.933333,Toulon,Provence,0,Marseille,43.3,5.4,Marseille,Provence,0,0,0.0,630.0,-5762707200000,5,May,21,Pierres,Autres et divers
"25,428",0000288N- 11,324,Brest,48.4,-4.483333,Brest,Bretagne,0,Bordeaux,44.833333,-0.566667,Bordeaux,Guyenne,-4,-1,60.0,94.0,-5744476800000,12,December,51,Lest,Sur lest et vides
"25,429",0000625N- 16,56,Rouen,49.433333,1.083333,Rouen,Normandie,0,Quillebeuf,49.466667,0.516667,Quillebeuf,Normandie,0,0,45.0,nan,-5754585600000,8,August,34,--,--
"25,430",0000591N- 15,142,Le Havre,49.5,0.133333,Le Havre,Normandie,0,Dunkerque,51.05,2.366667,Dunkerque,Flandre,-4,-1,36.0,nan,-5744476800000,12,December,51,--,--


In [37]:
# from pandas dataframe
#df_vaex = vaex.from_pandas(df=df, copy_index=False)

In [6]:
df1 = df_vaex_json[(df_vaex_json.departure.isin(["Bordeaux", "Le Havre"])) & (df_vaex_json.destination.isin(["La Rochelle"]))]

### Data manipulation

In [20]:
df2 = df_vaex_json[df_vaex_json.category_portic_fr.dropmissing()].groupby(['category_portic_fr','commodity_standardized_fr'],agg='count')
df2

#,category_portic_fr,commodity_standardized_fr,count
0,Bois divers et sous-produits du bois,Planche,258
1,Êtres humains,Passager,259
2,Autres produits alimentaires,Fruits sec,4
3,Céréales et légumineuses,Blé d' Inde,2
4,"'Combustibles, produits intermédiaires et autres...",Huile de poisson,3
...,...,...,...
244,Autres produits alimentaires,Beurre,54
245,Autres produits alimentaires,Boeuf salé,6
246,Produits textiles,Toile,28
247,Autres produits alimentaires,Vivres,5


In [17]:
df3 = df1.groupby(by=['departure_province','departure_admiralty']).agg({"tonnage":["mean","sum"]})
df3

#,departure_province,departure_admiralty,tonnage_mean,tonnage_sum
0,Normandie,Le Havre,76.7368,1458
1,Guyenne,Bordeaux,41.9077,2724


In [22]:
df4 = df_vaex_json.groupby(df_vaex_json.departure, progress='widget').agg(
    {'0': vaex.agg.count(df_vaex_json.travel_id, selection=df_vaex_json.departure_uncertainity == 0),
    '-1': vaex.agg.count(df_vaex_json.travel_id, selection=df_vaex_json.departure_uncertainity == -1),
    '-2': vaex.agg.count(df_vaex_json.travel_id, selection=df_vaex_json.departure_uncertainity == -2),
    '-3': vaex.agg.count(df_vaex_json.travel_id, selection=df_vaex_json.departure_uncertainity == -3),
    '-4': vaex.agg.count(df_vaex_json.travel_id, selection=df_vaex_json.departure_uncertainity == -4),
    '-5': vaex.agg.count(df_vaex_json.travel_id, selection=df_vaex_json.departure_uncertainity == -5),
    'total': vaex.agg.count(df_vaex_json.travel_id)}
).sort('total', ascending=False)

In [7]:
df4 = df_vaex_json.groupby(df_vaex_json.commodity_standardized_fr).agg(
    {f'{product}': vaex.agg.count(df_vaex_json.travel_id, selection=df_vaex_json.commodity_standardized_fr == f'{product}') for product in products_list}
)
df4

#,commodity_standardized_fr,Agrès,None,Fruits frais,Cornes de mouton,Filet,Passager (français),Sel,Blé,Cendre de varech,Seigle,Effets,Millet,Hareng,Savon,Cuir,Pois chiches,Amarinage,Poudre,Boeuf salé,Huile de morue,Sapin,Cuivre (pain),Lest,Pomme de terre,Pomme,Eau,Réglisse (bois),Effets de maison,"Alpiste, escayole",Prunes,Sardine préssée,Pois,Huile,Liège,Maquereau,Amidon,Graine de lin,"Brai, poix",Cidre,Campeche,Ardoise,Bois de teinture,"Grignons, marc d' olive",Cabotage,Cuivre (vieux),Boeuf,Bois à brûler,Cannelle,Pêche au hareng,Ecorce,Vin (bouteille),Filets de pêche,Fromage,Hareng en saumure,Dame-jeanne,Charbon,Oignons,"Fanon de baleine, barbes de baleine",Pêche à la sardine,Anchois,Haricots,Thé,Articles de parfumerie,"Poudre d' artillerie, poudre à feu",Tabac (feuille),Poudre à canon,Planches de sapin,Effets de naufrage,Acajou,Café Bourbon,Boulet,Matures,Denrées des colonies,Clou,Moussline,Cuivre (planches),Droguerie,Verre,Feuillard,Pavés de pierre,"Peaux, pelleteries",Cire,Potasse,Huile de cachalot,Fruits,Goudron,Pin,Vinaterie,Viande de pate,Miel,Bois de construction,Chargé,Orge,Amandes,Plomb,Morue sèche,Effets de pêche,Pêche aux moules,Verdet / Vert de gris,...,Vin espagnol,Lège,Jambon,Son,Filets (vieux),Morue,Biscuit,Fer blanc,Jarres (vides),Charbon de terre,Vesces,Mélasses,Raisin sec,Genièvre,Café (Moka),Tourbe,Citron,Toile,Tabac (poudre),Sucre,Terre,Sardine,Café,Térébenthine (huile),Oranges portuguaises,Chanvre,Vinaigre,Bière,Cuivre,Lard,Effets militaires,Vin muscat,Pierre à chaux,Poisson salé,Douelle,Hareng saur,Beurre,Merrain,Sardine salée,Sassafras,Pêche au maquereau,Chaux,Douve,Graines,Vin blanc,Laine,Panier,Marbre,Croisière des douanes,Chapeau de paille,Planche,Froment,Fèves,Peau de boeuf,Poisson,Réglisse,Pêche aux huitres,Chataignes,Eau de vie d' Hendaye,"Divers, assortiment",Morue verte,Avoine,Saumon,Hardes,Fumier,Fer en barre,Figue,Tabac,Blé d' Inde,Bois,Eau de vie,Poisson sec,Plâtre,Terre de Broue,Thé de l' Inde,Bouteilles,Grains,Riz,Etoupe,Provisions,Passager,Cacao,Fagots,Souffre,Bois de chauffage,Cendres,Moules,Drap,Porcelaine,Trafiquer,Fourrage,Oranges,Huitres,Plombs de chasse,Vin de Madère,Soude,Moutarde,Semoule,Diverses marchandises autorisées,Etoffe
0,Agrès,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,--,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Passager (français),0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Filet,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Fruits frais,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [23]:
df4.to_arrow_table()

pyarrow.Table
departure: large_string
0: int64
-1: int64
-2: int64
-3: int64
-4: int64
-5: int64
total: int64
----
departure: [["Rouen","Le Havre","La Rochelle","Bordeaux","Marennes",...,"La Mailleraye","Concarneau","Port-des-Barques","Cargèse ","Luçon"]]
0: [[1852,1737,1165,1103,1127,...,0,0,0,0,0]]
-1: [[7,3,2,2,0,...,0,0,0,0,1]]
-2: [[9,10,3,1,0,...,1,1,0,0,0]]
-3: [[5,2,18,3,1,...,0,0,0,0,0]]
-4: [[97,144,61,82,24,...,0,0,1,1,0]]
-5: [[5,1,27,11,0,...,0,0,0,0,0]]
total: [[1975,1897,1276,1202,1152,...,1,1,1,1,1]]

In [24]:
df4["departure"]

Expression = departure
Length: 201 dtype: large_string (column)
----------------------------------------
  0             Rouen
  1          Le Havre
  2       La Rochelle
  3          Bordeaux
  4          Marennes
         ...         
196     La Mailleraye
197        Concarneau
198  Port-des-Barques
199           Cargèse
200             Luçon

In [5]:
products_list = df_vaex_json.commodity_standardized_fr.unique()

In [7]:
{f'{product}': vaex.agg.count(df_vaex_json.travel_id, selection=df_vaex_json.commodity_standardized_fr == f'{product}') for product in products_list}

{'Savon': vaex.agg.count('travel_id'),
 'Fèves': vaex.agg.count('travel_id'),
 'Poudre à canon': vaex.agg.count('travel_id'),
 'None': vaex.agg.count('travel_id'),
 'Café': vaex.agg.count('travel_id'),
 'Soude': vaex.agg.count('travel_id'),
 'Canons': vaex.agg.count('travel_id'),
 'Plâtre': vaex.agg.count('travel_id'),
 'Fer': vaex.agg.count('travel_id'),
 'Blé': vaex.agg.count('travel_id'),
 'Cuivre': vaex.agg.count('travel_id'),
 'Laine': vaex.agg.count('travel_id'),
 'Orge': vaex.agg.count('travel_id'),
 'Meubles': vaex.agg.count('travel_id'),
 'Charbon de terre': vaex.agg.count('travel_id'),
 'Charbon de bois': vaex.agg.count('travel_id'),
 'Fagots': vaex.agg.count('travel_id'),
 'Goudron': vaex.agg.count('travel_id'),
 'Fumier': vaex.agg.count('travel_id'),
 'Pêche des moules': vaex.agg.count('travel_id'),
 'Huile': vaex.agg.count('travel_id'),
 'Pêche au poisson frais': vaex.agg.count('travel_id'),
 'Huitres': vaex.agg.count('travel_id'),
 'Hardes de mer': vaex.agg.count('travel_

In [8]:
df_vaex_json.groupby(df_vaex_json.outdate_fixed_datetime_month_number, progress='widget').agg(
    {f'{product}': vaex.agg.count(df_vaex_json.travel_id, selection=df_vaex_json.commodity_standardized_fr == f'{product}') for product in products_list}
)

#,outdate_fixed_datetime_month_number,Savon,Fèves,Poudre à canon,None,Café,Soude,Canons,Plâtre,Fer,Blé,Cuivre,Laine,Orge,Meubles,Charbon de terre,Charbon de bois,Fagots,Goudron,Fumier,Pêche des moules,Huile,Pêche au poisson frais,Huitres,Hardes de mer,Graines de lin,Moutarde,Lest,Morue verte,Marchandises permises,Suif,Pommes,Sucre,Morue,Diverses marchandises,Sardines,Charbon,Pois,Cendre de varech,Pierres,Bois de construction,Froment,Bois,Sel,Vide,Ardoises,Feuillard,Lentilles,Son,Etoupe,Poisson,Salaisons,Cendre,Brai,Lin,Vinaigre,Ustensiles de pêche,Poisson sec,Bouteilles,Terre de Broue [pour verrerie],Farine,Planches,Sardines pressées de Bretagne,Fruits,Avoine,Bois à brûler,Passagers,Terre,Biscuits,Vin,Pêche à la morue,Vivres,Beurre,Briques,Bois merrain,Moules,Hardes,Sardines salées,Poterie,Turbe,Cercles,Huile de poisson,Grain,Fromage,Résine,Fourrage,Futailles vides,Tuilles,Morue sèche,à l'aventure,Gréments de navire,Pêche à la sardine,Eau-de-vie,Haricots blancs,Chaux,Chanvre,Marchandises d'un naufrage,Bois de chauffage,Peaux de bœuf,Pierre de taille,Coton,Lard salé,Vinatérie,Seigle,Bière,Oignons,Verre cassé
0,1,0,3,0,0,0,14,0,7,4,18,0,0,1,0,5,2,5,2,0,0,18,19,5,0,6,0,265,0,0,2,0,4,7,57,5,6,0,2,4,13,22,28,147,125,0,0,0,2,0,0,1,2,3,0,1,0,0,4,0,7,16,0,1,4,30,11,0,0,157,0,0,2,2,3,0,0,0,0,0,0,0,14,4,9,1,33,0,2,0,0,0,23,1,5,1,0,10,0,7,1,0,0,0,1,2,0
1,2,0,4,1,0,0,1,0,5,6,28,1,1,4,3,3,1,7,0,2,0,24,25,15,0,11,0,192,3,0,0,0,1,3,53,0,3,1,0,4,7,9,20,120,59,0,3,0,0,0,0,2,0,0,0,1,0,0,6,0,17,18,0,0,2,22,21,0,0,147,7,0,0,1,2,0,1,0,1,0,0,0,10,1,15,0,30,0,1,0,1,0,24,1,5,0,2,14,0,1,0,0,0,0,0,0,0
2,3,2,0,1,0,1,18,0,7,6,22,0,0,2,1,12,2,5,2,3,0,36,12,24,0,4,0,257,1,1,1,0,7,1,55,1,13,0,0,14,22,15,28,193,70,0,5,0,1,0,0,0,1,4,1,4,3,0,6,0,15,21,0,1,0,27,20,0,0,163,34,0,0,0,3,6,1,0,4,0,1,0,8,0,14,3,47,0,0,0,0,0,18,0,15,0,0,17,0,0,0,0,2,2,1,0,0
3,4,2,1,1,0,0,11,0,5,1,14,1,1,0,2,4,0,5,1,3,1,16,4,4,0,1,0,283,0,0,0,0,4,1,40,0,9,0,0,4,13,6,24,262,82,0,6,0,1,0,0,0,4,3,1,0,2,0,3,0,13,16,0,0,1,31,24,0,0,125,2,0,4,1,4,2,0,0,0,2,0,0,10,0,4,0,34,1,0,0,0,0,13,0,19,0,0,6,0,0,0,0,4,0,2,0,0
4,5,2,1,0,0,0,7,3,3,9,16,1,1,1,2,10,1,1,2,1,0,16,8,0,1,0,0,261,0,0,0,0,2,1,24,0,6,0,2,28,20,6,27,231,71,0,8,0,2,1,0,0,6,0,2,3,0,0,3,0,15,33,0,0,6,25,27,3,0,126,0,0,1,2,6,0,0,0,0,0,0,1,8,1,13,2,34,4,0,1,0,0,12,0,15,3,2,13,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7,8,0,8,1,0,3,8,2,12,1,27,0,0,0,2,8,3,4,4,1,0,11,13,12,1,1,1,387,1,0,0,0,4,0,47,5,11,0,0,13,17,3,23,220,116,0,7,2,2,0,1,0,2,1,0,3,1,1,4,2,11,23,1,0,4,46,31,0,0,83,0,1,11,0,1,3,0,0,4,0,1,0,2,1,11,3,42,5,0,0,0,1,29,0,23,0,0,20,0,1,0,0,1,0,0,1,0
8,9,0,6,2,0,1,8,0,8,3,17,0,2,0,1,5,1,3,3,0,0,17,11,36,2,0,0,285,1,1,0,0,3,4,51,1,3,1,0,7,14,20,20,191,96,4,1,1,1,0,0,0,1,2,0,1,1,0,4,2,8,31,1,0,8,23,18,0,0,62,0,0,11,0,0,2,0,2,4,0,0,2,4,1,8,0,18,6,0,0,0,2,20,0,9,2,0,8,0,1,1,0,0,0,0,1,0
9,10,0,10,1,0,2,13,4,3,4,39,0,2,1,0,4,0,2,1,0,0,8,2,58,0,1,0,258,0,3,0,0,3,9,43,5,5,0,1,4,7,22,14,128,71,1,1,0,2,0,2,0,1,4,0,0,0,0,3,0,14,19,3,0,5,27,24,0,0,152,0,0,4,0,2,0,1,0,1,0,0,0,3,0,10,3,15,0,0,0,0,3,20,0,8,1,0,7,1,0,0,0,0,0,0,4,0
10,11,2,6,0,0,0,11,1,4,2,38,0,2,1,0,2,3,2,1,0,0,11,0,67,2,0,0,215,0,0,1,1,2,8,31,5,2,0,1,5,7,31,20,118,97,0,2,1,1,0,1,0,0,0,0,3,0,0,4,0,10,14,2,0,5,18,18,0,0,140,0,1,5,2,1,0,0,0,3,0,0,0,6,1,2,3,15,0,0,0,0,0,18,2,14,5,0,5,2,0,1,0,1,1,0,1,0


In [11]:
df_vaex_json[df_vaex_json.commodity_standardized_fr.notna()].travel_id.count()

array(13548, dtype=int64)